In [ ]:
%matplotlib inline

In [ ]:
import numpy as np

# soft thresholding function
def soft(z, th):
    return np.sign(z) * np.maximum(np.abs(z) - th, 0)

In [ ]:
from scipy import linalg

# Iterative soft thresholding (IST)
def IST(A, b, tol=1e-5, maxiter=300, l=1., L=None):
    m, n = A.shape
    x = np.zeros(n)
    r = b.copy()
    if L is None:    # roughly estimate the Lipschitz constant
        L = 2*linalg.norm(A.dot(A.T.dot(b))) / linalg.norm(b)
    count = 0
    cost_history = []
    normr = linalg.norm(r)
    while count < maxiter and normr > tol:
        count += 1
        x = soft(x + A.T.dot(r) / L, l/L)
        r = b - A.dot(x)
        normr = linalg.norm(r)
        cost_history.append( normr*normr + l*np.sum(np.abs(x)) )
    return x, cost_history

In [ ]:
from time import time
# Generate sythetic data for demo
rng = np.random.RandomState(int(time()))
#m, n = 512, 2048
m, n = 1024, 8192
#m, n = 2000, 4000

# use a random matrix as a basis (design matrix)
A = rng.randn(m, n) / np.sqrt(m)

# generate a k-sparse Gaussian signal vector
k = 100
stdx = 1.
snr = 10.

x_true = np.zeros(n)
T = np.sort(rng.choice(n,k,replace=False))
print('True support of %d nonzeros = ' % (k))
print(T)
x_true[T] = rng.randn(k) * stdx

# make the query vector
b = A.dot(x_true)

# add noise
normb = linalg.norm(b)
noise = rng.randn(m)
noise = noise / linalg.norm(noise) * normb / snr
tol = linalg.norm(noise)
b = b + noise

In [ ]:
print("Running IST..")
t0 = time()
l = (stdx*stdx / k * m) / np.sqrt(snr) / normb
x_est, ch = IST(A, b, tol=tol, l=l, maxiter=300)
print('done in %.2fs.' % (time() - t0))

import matplotlib.pyplot as plt
plt.loglog(ch)

Tpred = np.nonzero(x_est)[0]
print('Predicted supprt of %d nonzeros = ' % (np.count_nonzero(x_est)))
print(Tpred)

from sklearn import metrics
print(metrics.classification_report(x_true == 0, x_est == 0))
print(metrics.confusion_matrix(x_true == 0, x_est == 0))

In [ ]:
print('rel. error = %.2e' % (linalg.norm(x_est-x_true)/linalg.norm(x_true)))
import matplotlib.pyplot as plt
plt.figure()
plt.plot(np.arange(n), x_true, 'g.', markersize=8, mec='green', label='True')
plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', mew=1, label='Estimated')
plt.legend(loc='upper right', shadow=False)
plt.show()

In [ ]:
# debias by least squares
Tpred = np.nonzero(x_est)[0]
x_est[Tpred] = np.linalg.lstsq(A[:,Tpred], b, rcond=None)[0]

print('rel. error = %.2e' % (linalg.norm(x_est-x_true)/linalg.norm(x_true)))
import matplotlib.pyplot as plt
plt.figure()
plt.plot(np.arange(n), x_true, 'g.', markersize=8, mec='green', label='True')
plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', mew=1, label='Estimated')
plt.legend(loc='upper right', shadow=False)
plt.show()